# 如何使用mysql_tool操作資料庫


在本篇中你會瞭解到:
1. 如何使用staff_tab_portal連接資料庫
2. 如何使用crawler_tool查詢資料表的資料
3. 如何使用mysql_tool插入資料到資料表
4. 如何使用mysql_interface建立連接界面

在開始之前請先確認你有MySQL Server可以使用，如果想要自己架設的話可以參考這篇:[Install_MySQL_Server_on_the_kubuntu](https://github.com/jimmyzzzz/Install_MySQL_Server_on_the_kubuntu)

全篇中會使用到的MySQL Server資料庫的設定如下:
* HOST: 192.167.0.103
* 用戶名: testuser
* 密碼: 123
* 資料庫: stock_db
* 資料表: staff_tab, twse_pv

***

## 連接資料庫

我們可以使用*staff_tab_portal*連接資料庫*stock_db*中的資料表*staff_tab*，你會發現我們只需要輸入用戶名跟密碼，剩下的部份已經幫你預設好了。

In [1]:
from mysql_tool import staff_tab_portal

staff_tab=staff_tab_portal(user='testuser', password='123')

同理我們如果需要連接資料表*twse_pv*只需要呼叫對應的類*staff_tab_portal*即可。

In [2]:
from mysql_tool import twse_pv_portal

twse_tab=twse_pv_portal(user='testuser', password='123')

連接資料後，會回傳一個資料表的操作界面。使用者可以使用該界面做查詢資料或插入資料等MySQL操作。

***

## 查看資料表資訊

在使用資料表時，不免有時要查看資料的一些基本資料，例如:欄位或是資料表的資料筆數等等。如果使用*mysql_tool*可以使用下面的方式查看這些基本資料:

查看資料表有哪些欄位:

In [19]:
staff_tab.columns

['ID', 'birthday', 'age', 'tall']

查看資料表欄位的詳細資訊:

In [21]:
staff_tab.columns_info

,Field,Type,Null,Key,Default,Extra
0,ID,int,NO,PRI,None,auto_increment
1,birthday,date,YES,,None,
2,age,int,YES,,None,
3,tall,"decimal(9,3)",YES,,None,


查看資料表有幾筆資料:

In [23]:
len(staff_tab)

7

***

## 查詢資料表

要查看資料表的內容可以像使用*list*或*dict*一樣在中括號中輸入查詢的內容就會回傳相應的*DataFrame*資料。

查詢全部資料:

In [24]:
staff_tab[:]

,ID,birthday,age,tall
0,1,2022-01-01,18.0,160.16
1,2,2022-01-02,19.0,NaN
2,3,2022-01-03,NaN,180.18
3,4,2022-01-04,21.0,118.01
4,5,2022-01-05,22.0,178.00
5,6,2022-01-06,23.0,160.00
6,7,2022-01-07,NaN,NaN


查詢第1筆資料:

In [3]:
staff_tab[0]

,ID,birthday,age,tall
0,1,2022-01-01,18,160.16


查詢前3筆資料:

In [4]:
staff_tab[:3]

,ID,birthday,age,tall
0,1,2022-01-01,18.0,160.16
1,2,2022-01-02,19.0,NaN
2,3,2022-01-03,NaN,180.18


查詢第2筆到最後一筆的資料:

In [5]:
staff_tab[1:-1]

,ID,birthday,age,tall
0,2,2022-01-02,19.0,NaN
1,3,2022-01-03,NaN,180.18
2,4,2022-01-04,21.0,118.01
3,5,2022-01-05,22.0,178.00
4,6,2022-01-06,23.0,160.00


查詢*birthday*欄位的資料:

In [6]:
staff_tab['birthday']

,birthday
0,2022-01-01
1,2022-01-02
2,2022-01-03
3,2022-01-04
4,2022-01-05
5,2022-01-06
6,2022-01-07


查詢*ID*欄位和*birthday*欄位資料:

In [7]:
staff_tab[['ID','birthday']]

,ID,birthday
0,1,2022-01-01
1,2,2022-01-02
2,3,2022-01-03
3,4,2022-01-04
4,5,2022-01-05
5,6,2022-01-06
6,7,2022-01-07


***

## 有條件的查詢

有時候我們會希望查詢的時候使用到條件這時候就可以用*columns_condition*來產生查詢條件，比如說你想查詢生日在2022-01-03之後的資料:

In [8]:
from mysql_tool import columns_condition as col

condition1=col['birthday']>'2022-01-03'
staff_tab[condition1]

,ID,birthday,age,tall
0,4,2022-01-04,21.0,118.01
1,5,2022-01-05,22.0,178.00
2,6,2022-01-06,23.0,160.00
3,7,2022-01-07,NaN,NaN


你也可以使用操作屬性的方式來查詢，比如你想查詢歲數小於22歲的。

In [9]:
condition2=col.age<22
staff_tab[condition2]

,ID,birthday,age,tall
0,1,2022-01-01,18,160.16
1,2,2022-01-02,19,NaN
2,4,2022-01-04,21,118.01


你也可以將多個條件同時使用:

In [10]:
staff_tab[condition1&condition2]

,ID,birthday,age,tall
0,4,2022-01-04,21,118.01


In [11]:
staff_tab[condition1|condition2]

,ID,birthday,age,tall
0,1,2022-01-01,18.0,160.16
1,2,2022-01-02,19.0,NaN
2,4,2022-01-04,21.0,118.01
3,5,2022-01-05,22.0,178.00
4,6,2022-01-06,23.0,160.00
5,7,2022-01-07,NaN,NaN


可以使用類似DataFrame中的*isin*，如果是取反則是使用*noin*:

In [12]:
staff_tab[col.age.isin([18,19,21])]

,ID,birthday,age,tall
0,1,2022-01-01,18,160.16
1,2,2022-01-02,19,NaN
2,4,2022-01-04,21,118.01


In [13]:
staff_tab[col.age.noin([18,19,21])]

,ID,birthday,age,tall
0,5,2022-01-05,22,178.0
1,6,2022-01-06,23,160.0


搜索為*NULL*的值可以用*is_null*，取反則使用*no_null*:

In [14]:
staff_tab[col.age.is_null()]

age IS NULL


,ID,birthday,age,tall
0,3,2022-01-03,None,180.18
1,7,2022-01-07,None,NaN


In [15]:
staff_tab[col.age.no_null()]

,ID,birthday,age,tall
0,1,2022-01-01,18,160.16
1,2,2022-01-02,19,NaN
2,4,2022-01-04,21,118.01
3,5,2022-01-05,22,178.00
4,6,2022-01-06,23,160.00


如果真的需要使用MySQL指令查詢資料，可以用下面的方式。

In [26]:
staff_tab.read_df('SELECT * FROM staff_tab')

,ID,birthday,age,tall
0,1,2022-01-01,18.0,160.16
1,2,2022-01-02,19.0,NaN
2,3,2022-01-03,NaN,180.18
3,4,2022-01-04,21.0,118.01
4,5,2022-01-05,22.0,178.00
5,6,2022-01-06,23.0,160.00
6,7,2022-01-07,NaN,NaN


In [17]:
staff_tab.fetch('SELECT * FROM staff_tab')

((1, datetime.date(2022, 1, 1), 18, Decimal('160.160')),
 (2, datetime.date(2022, 1, 2), 19, None),
 (3, datetime.date(2022, 1, 3), None, Decimal('180.180')),
 (4, datetime.date(2022, 1, 4), 21, Decimal('118.010')),
 (5, datetime.date(2022, 1, 5), 22, Decimal('178.000')),
 (6, datetime.date(2022, 1, 6), 23, Decimal('160.000')),
 (7, datetime.date(2022, 1, 7), None, None))

In [18]:
staff_tab.fetch('SELECT * FROM staff_tab', n=5)

((1, datetime.date(2022, 1, 1), 18, Decimal('160.160')),
 (2, datetime.date(2022, 1, 2), 19, None),
 (3, datetime.date(2022, 1, 3), None, Decimal('180.180')),
 (4, datetime.date(2022, 1, 4), 21, Decimal('118.010')),
 (5, datetime.date(2022, 1, 5), 22, Decimal('178.000')))

***

## 插入資料

與查訊資料相對的如何插入資料到資料表中。如果要插入資料可以使用*insert_df*將DataFrame插入到資料表中:

In [30]:
import pandas as pd
df=pd.DataFrame({'birthday':['2022-01-07'],'age':[19],'tall':[167.67]})
staff_tab.insert_df(df)

查看最後一筆資料會發現已經將*df*加入到資料庫中。

In [33]:
staff_tab[-1]

,ID,birthday,age,tall
0,8,2022-01-07,19,167.67


如果需要使用MySQL指令的方式插入資料則可以使用*commit*方法插入資料:

In [35]:
cmd_list=[
    "INSERT INTO staff_tab(birthday, age, tall) VALUES('2022-01-08', 20, 194)"
]
staff_tab.commit(cmd_list=cmd_list)

查看最後一筆資料會發現已經將資料加入到資料庫中。

In [36]:
staff_tab[-1]

,ID,birthday,age,tall
0,9,2022-01-08,20,194.0


**`補充:`** commit方法

使用*commit方法*不僅僅可以用來插入資料，還可以對資料表做其他任何MySQL的操作。與*fetch*方法不同的地方在於一個會回傳資料庫的回傳結果，一個不會。

***

## 使用mysql_interface建立連接工具

前面一開始我們使用了*staff_tab_portal*這個連接工具連接到了*staff_tab*資料表。那麼如股果我們要連接其他資料表的時候能不能製作自己的連接工具(portal)呢？我們可以使用mysql_tool.mysql_interface來建立。

順帶一提，mysql_tool.mysql_portal中的所有連接工具也是使用mysql_interface來建立的。

In [43]:
from mysql_tool import mysql_interface
import pymysql

class twse_tab_portal(mysql_interface):
    def __init__(self, user, password):
        bd_name='stock_db'
        tab_name='twse_pv'
        conn=pymysql.connect(
            host='192.168.0.103',
            port=3306,
            user=user,
            password=password,
            db='stock_db'
        )
        
        super().__init__(conn=conn, tab=tab_name)

上面是一個使用*mysql_interface*建立連接工具的範例，在範例中建立了一個與資料庫*stock_db*中的資料表*twse_pv*的連接工具。

如果想要使用連接工具建立連接，可以輸入帳密，然後該連接工具會回傳一個資料表的操作界面。然後就可以像上面介紹過的一樣操作資料表了。

In [45]:
twse_tab=twse_tab_portal('testuser', '123')

# 查訊資料表欄位資訊
twse_tab.columns_info

,Field,Type,Null,Key,Default,Extra
0,ID,int,NO,PRI,None,auto_increment
1,date,date,YES,,None,
2,stock_id,varchar(32),YES,,None,
3,open,"decimal(9,3)",YES,,None,
4,high,"decimal(9,3)",YES,,None,
5,low,"decimal(9,3)",YES,,None,
6,close,"decimal(9,3)",YES,,None,
7,volume,int,YES,,None,
8,value,int,YES,,None,
9,record,int,YES,,None,
